# Web Mining and Applied NLP (44-620)

## Final Project: Article Summarizer

### Student Name: **Caleb Sellinger**

### GitHub Repo [HERE](https://github.com/crsellinger/article-summarizer)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

You should bring in code from previous assignments to help you answer the questions below.

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

In [27]:
from collections import Counter
import pickle
import requests
import spacy
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import selenium

!pip list

print('All prereqs installed.')

Package                 Version
----------------------- -----------
annotated-types         0.7.0
asttokens               3.0.0
attrs                   25.3.0
beautifulsoup4          4.13.4
blis                    1.3.0
catalogue               2.0.10
certifi                 2025.8.3
cffi                    1.17.1
charset-normalizer      3.4.2
click                   8.2.1
cloudpathlib            0.21.1
colorama                0.4.6
comm                    0.2.3
confection              0.1.5
contourpy               1.3.3
cycler                  0.12.1
cymem                   2.0.11
debugpy                 1.8.15
decorator               5.2.1
en_core_web_sm          3.8.0
executing               2.2.0
fonttools               4.59.0
h11                     0.16.0
idna                    3.10
ipykernel               6.30.1
ipython                 9.4.0
ipython_pygments_lexers 1.1.1
jedi                    0.19.2
Jinja2                  3.1.6
jupyter_client          8.6.3
jupyter_core      

1. Find on the internet an article or blog post about a topic that interests you and you are able to get the text for using the technologies we have applied in the course.  Get the html for the article and store it in a file (which you must submit with your project)

In [28]:
# Webpage uses JavaScript to load content, need Selenium to run JS to pull content
# BS4 does not run scripts and was only pulling root div with scripts
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://www.kaggle.com/datasets/sudan007kaggler/reddit-rwallstreet-bets-posts-dataset-labelled/data"

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Load the page
driver.get(url)

try:
    # Wait for the specific div with role="rowgroup" to be present
    # Wait a max of 20 seconds.
    wait = WebDriverWait(driver, 20)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="rowgroup"]')))
    
    # Get the page source after JavaScript has loaded
    html_content = driver.page_source
    
    # Now, parse the fully loaded HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find table
    element = soup.find('div', attrs={'role': 'rowgroup'})

    if element:
        with open('content.pkl','wb') as file:
            pickle.dump(element.text, file)
        # print(element.prettify)
        with open('content.pkl','rb') as file:
            content = pickle.load(file)
        print(content)
    else:
        print("Could not find the div with role='rowgroup' even after waiting.")

finally:
    # Close browser window
    driver.quit()


they still aren covering0hold your fucking stocks0party on dudes1something shady going on04 00am pre market hype0diamond hands relax as my 3d printer sings you the song of it people army1dogeeeee0rkt dividends but not because of the dividends0i used all my stimmy0zom looks good to go1so basically am moonkey0mertgag ageddon uwmc finally the play1doing my part to show the market how meaningless robinhood is-1aal american airlines year long consolidation breakout cheers to the moom and beyond1proud of my holdings m down nothing comparing to big boys but if ever single one in the page sell ll...-1january 28 2021 7 14am pst never forget1webull0even htmw won let me trade and it fake money1put all in on blackberry dip0market level 2 data0uk brothers and sisters freetrade are still allowing gme bb amc trading and instant account topups o...0counterfeit shares and what we should be doing right now1public won let buys on the dip1unite against the machine0you ve won there are no evil doers left o

<div role="rowgroup"><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>they still aren covering</td><td>0</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>hold your fucking stocks</td><td>0</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>party on dudes</td><td>1</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>something shady going on</td><td>0</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>4 00am pre market hype</td><td>0</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>diamond hands relax as my 3d printer sings you the song of it people army</td><td>1</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr class="sc-ilnnPk sc-hfsVOy dSkrLE hnoKRX"><td>dogeeeee</td><td>0</td></tr></span><span class="sc-iMGFoU iLsOEd"><tr

2. Read in your article's html source from the file you created in question 1 and do sentiment analysis on the article/post's text (use `.get_text()`).  Print the polarity score with an appropriate label.  Additionally print the number of sentences in the original article (with an appropriate label)

3. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent tokens (converted to lower case).  Print the common tokens with an appropriate label.  Additionally, print the tokens their frequencies (with appropriate labels)

4. Load the article text into a trained `spaCy` pipeline, and determine the 5 most frequent lemmas (converted to lower case).  Print the common lemmas with an appropriate label.  Additionally, print the lemmas with their frequencies (with appropriate labels).

5. Make a list containing the scores (using tokens) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores. From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

6. Make a list containing the scores (using lemmas) of every sentence in the article, and plot a histogram with appropriate titles and axis labels of the scores.  From your histogram, what seems to be the most common range of scores (put the answer in a comment after your code)?

7. Using the histograms from questions 5 and 6, decide a "cutoff" score for tokens and lemmas such that fewer than half the sentences would have a score greater than the cutoff score.  Record the scores in this Markdown cell

* Cutoff Score (tokens): 
* Cutoff Score (lemmas):

Feel free to change these scores as you generate your summaries.  Ideally, we're shooting for at least 6 sentences for our summary, but don't want more than 10 (these numbers are rough estimates; they depend on the length of your article).

8. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on tokens) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

9. Print the polarity score of your summary you generated with the token scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

10. Create a summary of the article by going through every sentence in the article and adding it to an (initially) empty list if its score (based on lemmas) is greater than the cutoff score you identified in question 8.  If your loop variable is named `sent`, you may find it easier to add `sent.text.strip()` to your list of sentences.  Print the summary (I would cleanly generate the summary text by `join`ing the strings in your list together with a space (`' '.join(sentence_list)`).

11. Print the polarity score of your summary you generated with the lemma scores (with an appropriate label). Additionally, print the number of sentences in the summarized article.

12.  Compare your polarity scores of your summaries to the polarity scores of the initial article.  Is there a difference?  Why do you think that may or may not be?.  Answer in this Markdown cell.  

13. Based on your reading of the original article, which summary do you think is better (if there's a difference).  Why do you think this might be?